In [ ]:
'''
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
'''

In [ ]:
#!git clone https://github.com/google-research/simclr.git

In [ ]:
'''
import matplotlib.pyplot as plt

plt.figure()
train_sample = plt.imread('/kaggle/input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
plt.subplot(121)
plt.imshow(train_sample)
test_sample = plt.imread('/kaggle/input/cassava-leaf-disease-classification/test_images/2216849948.jpg')
plt.subplot(122)
plt.imshow(test_sample)
print(train_sample.shape)
print(test_sample.shape)
'''

In [ ]:
'''
from PIL import Image
import matplotlib.pyplot as plt
plt.figure()
sample = Image.open('/kaggle/input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
plt.subplot(121)
plt.imshow(sample)
sample_resize = sample.resize((600, 600))
plt.subplot(122)
plt.imshow(sample_resize)
'''

データの集計

In [ ]:
'''
import pandas as pd

train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train.head()
'''

In [ ]:
#train.label.value_counts()

label count
* 0: 1087
* 1: 2189
* 2: 2386
* 3: 13158
* 4: 2577

In [ ]:
#!python /kaggle/working/simclr/tf2/run.py --train_mode=pretrain --train_batch_size=256 --train_epochs=1 --dataset=cifar10 --image_size=32 --resnet_depth=18 --model_dir=/kaggle/working/simclr/tf2/cache --eval_split=test --use_tpu=False

* 上のセルの実行でGPUのRAMは15.2GB/15.9GB使用しておりかなりギリギリ
* CPUもGPUも使用率100%になっていた

# **入力の形式**

In [ ]:
'''
import tensorflow_datasets as tfds
import tensorflow as tf
builder = tfds.builder('cifar10', data_dir='/kaggle/working/data')
builder.download_and_prepare()
print('num_train_examples:', builder.info.splits['train'].num_examples)
print('num_eval_examples: ', builder.info.splits['test'].num_examples)
print('num_classes: ', builder.info.features['label'].num_classes)
dataset = builder.as_dataset(as_supervised=True) # これにより辞書型ではなくなる
train_dataset, test_dataset = dataset['train'], dataset['test']
assert isinstance(train_dataset, tf.data.Dataset)
'''

In [ ]:
#print(train_dataset)

In [ ]:
'''
def map_fn(image, label):
      """Produces multiple transformations of the same batch."""
      print(type(image))
      label = tf.one_hot(label, 10)
      return image, label
'''

In [ ]:
#train_dataset = train_dataset.map(map_fn)
#train_dataset

# **cassavaのデータを上の形式に合わせる**

In [ ]:
'''
import pandas as pd

train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train.head()
'''

In [ ]:
#from sklearn.model_selection import train_test_split
#train, test = train_test_split(train, stratify=train['label'])

In [ ]:
#train

In [ ]:
#test

In [ ]:
'''
all_image_paths = list(train['image_id'])
all_image_paths = ['../input/cassava-leaf-disease-classification/train_images/' + str(path) for path in all_image_paths]
all_image_labels = list(train['label'])
'''

In [ ]:
#print(all_image_paths[:5])
#print(all_image_labels[:5])

In [ ]:
'''
import tensorflow as tf
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [600, 600])
  image = tf.cast(image, tf.uint8)
  #image /= 255.0  # normalize to [0,1] range
  return image

def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)
'''

In [ ]:
'''
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
print(image_label_ds)
'''

In [ ]:
'''
BATCH_SIZE = 32
image_count = len(all_image_paths)
ds = image_label_ds.shuffle(buffer_size=image_count)
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
'''
# `prefetch`を使うことで、モデルの訓練中にバックグラウンドでデータセットがバッチを取得できます。
#ds = image_label_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
#print(ds)

In [ ]:
# 画像をnormalizeしないと表示できない
'''
import matplotlib.pyplot as plt

plt.figure(figsize=(6,6))
for n,image in enumerate(image_ds.take(2)):
  plt.subplot(1,2,n+1)
  plt.imshow(image)
plt.show()
'''

# **辞書型でなくても動くかの確認**

In [ ]:
'''
def map_fn(image, label):
      """Produces multiple transformations of the same batch."""
      print(type(image))
      label = tf.one_hot(label, 4)
      return image, label
'''

In [ ]:
#ds = ds.map(map_fn)

In [ ]:
#ds

そもそもcifar10も辞書型ではなくなることが判明したので大丈夫

# CassavaデータでSimCLRを検証(12/21)

In [ ]:
#!git clone https://github.com/ta9ryuWalrus/simclr.git

In [ ]:
#!python /kaggle/working/simclr/tf2/run.py --train_mode=pretrain --train_batch_size=32 --train_epochs=5 --resnet_depth=18 --model_dir=/kaggle/working/trained_model --eval_split=test --use_tpu=False

* 上記の設定でGPUのメモリーは15.2GB/15.9GB使う
* GPU使用率の上限に関わらずメモリの使用量は一定なので、無駄な読み込みが発生していそう(?)
* またGPU使用率は0~100%で周期的に変動しているので、iterationの切れ目とかCPUの処理が律速になってしまっている可能性がある(CPU使用率は常に150%前後(←?))
* CPUのメモリやDiskはほとんど使われていない
* batch size32は動く(48は動かない)
* batch size32だと15分くらいで1epochが終わる
* 試しに5epochやってみたが精度は向上せず->pretrainなのでそれは問題ない(寧ろlabelを一切使わずに60%くらい当ててるのすごい)
* contrast accがどれくらい上がるまで学習すべきなのか？5epochでは15%程度しか当てられていない

In [ ]:
#!python /kaggle/working/simclr/tf2/run.py --mode=train_then_eval --train_mode=finetune --train_batch_size=32 --train_epochs=10 --resnet_depth=18 --model_dir=/kaggle/working/trained_model --eval_split=test --use_tpu=False

* top1 accuracy: 0.61
* top5 accuracy: 1.0(5classなので当たり前)

* trainとtestの分け方とlabeledとunlabeledの分け方が違うっぽい。
* train:testを1:9にしたら上記のようになったが、eval時に全体の9割を占めるtestのlabel情報を使ってfine-tuningしてしまっている気がする
* labeledとunlabeledの分け方はどこで決めている？
* ↑data.pyの_input_fnにあるbuilder.as_dataset(as_supervised=True)が怪しい気がする->as_supervisedは辞書型からimageとlabelのtupleに変換しているだけ

# ***log***
* pretrain5epochでaccが60％くらいになり
* finetuning10epochで70%弱くらいまで上昇

# データの使い方の方針
* trainとtestに分ける(7:3とか)
* pretrainではlabelは使わないのでtrainをそのまま使う
* fine-tuningの際にlabelを使うが、ここではsemi-supervisedの性能を検証したいので、trainのうち10%程度のみをfine-tuningに使う

こういう感じ？
* あくまでもSimCLRはself-supervisedの手法なので、pretrainのところしかちゃんと記述されていない感
* その後はsemi-supervisedとして使うも、別のデータに対してsupervisedでfine-tuningするも、各々の自由という感じか？

# ***課題***
* 学習済みモデルの保存方法
* /kaggle/working/の下にモデルを保存するようにはなっているが、notebookを再起動すると消えてしまう

# ***学習率とoptimizerを変更してみる***

In [ ]:
#!python /kaggle/working/simclr/tf2/run.py --learning_rate=1e-3 --optimizer=adam --train_mode=pretrain --train_batch_size=32 --train_epochs=40 --resnet_depth=18 --model_dir=/kaggle/working/trained_model --eval_split=test --use_tpu=False

* 10epochの学習でcontrast accが26.6%まで向上した
* supervised accは相変わらず61%程度
* 15epoch学習すると、contrast accが41.6%、supervised accが61.4%
* 上のコードでpretrainしたモデルがtrained_modelに保存されている

In [ ]:
#!python /kaggle/working/simclr/tf2/run.py --learning_rate=1e-3 --optimizer=adam --mode=train_then_eval --train_mode=finetune --train_batch_size=32 --train_epochs=10 --resnet_depth=18 --model_dir=/kaggle/working/trained_model --eval_split=test --use_tpu=False

* 15epochのpretrainの後に20epochのfine-tuning
* 学習データに対する正解率は66.2%でテストデータに対する正解率は62.0%(あまり学習できていない？)

## 1/13

In [ ]:
#! git clone https://github.com/ta9ryuWalrus/simclr.git

* 40epoch pretrainしたところ、contrast accが0.745、supervised accが0.621となった
* 学習済みモデルを保存できた

* 40epoch事前学習したモデルを読み込んでfinetuningしてみる
* inputディレクトリにdatasetとして学習済みモデルをloadしているが、permissionの関係でここから直接読み込んでfinetuningすることができなかったので、一度outputの方に学習済みモデルをコピーしてからfinetuningしている

In [ ]:
#! cp -r ../input/trained-model/trained_model /kaggle/working/trained_model

40epoch事前学習したやつをさらに追加で30epoch事前学習する
-> 古いモデルは削除されるようなコードになっていたので、残したければ古いやつと新しく保存するやつを別ディレクトリに残す必要がありそう(ここを書き換えるのは面倒)

In [ ]:
#!python /kaggle/working/simclr/tf2/run.py --learning_rate=1e-3 --optimizer=adam --train_mode=pretrain --train_batch_size=32 --train_epochs=70 --resnet_depth=18 --model_dir=/kaggle/working/trained_model --eval_split=test --use_tpu=False

fine-tuning

In [ ]:
#! python /kaggle/working/simclr/tf2/run.py --learning_rate=1e-3 --optimizer=adam --mode=train_then_eval --train_mode=finetune --train_batch_size=32 --train_epochs=10 --pretrain_steps=19188 --resnet_depth=18 --model_dir=/kaggle/working/trained_model --eval_split=test --use_tpu=False

* 学習率やepoch数などを変えてもaccが0.622から全然変わらないのでfinetuningされていない可能性
* logを読むに学習済みモデルの読み込みは正しく行われていそう
* global step=19188というのがpretrainから引き継がれていて、finetuningのtrain_steps=469を上回ってしまっていることから学習が行われずにいる。
* -> finetuning時のepoch数はpretrainのepoch数+finetuningのepoch数として指定する？->結局finetuningのデータ数が少なくstep数が小さく見積もられるのでこれではダメ
* -> 書き換えた。--pretrainでpretrainで学習したstep数を指定することで、train_stepsを正しく計算できる。--train_epochsのところはfinetuningを行うepoch数を指定する。
* とりあえず期待通りに動くことは確認できた
* 40epochのpretrainのあと10epochのfinetuningをしたときの精度は0.628程度(finetuning途中のtrainデータに対する精度は0.68くらい)

### pretrainを少し短めにした場合 ###

In [ ]:
#! python /kaggle/working/simclr/tf2/run.py --learning_rate=1e-4 --optimizer=adam --train_mode=pretrain --train_batch_size=32 --train_epochs=10 --resnet_depth=18 --model_dir=/kaggle/working/trained_model2 --eval_split=test --use_tpu=False

* lr=1e-4、epoch=10でpretrain
* contrast_acc=0.170、supervised_acc=0.614

In [ ]:
#! python /kaggle/working/simclr/tf2/run.py --learning_rate=1e-4 --optimizer=adam --mode=train_then_eval --train_mode=finetune --train_batch_size=32 --train_epochs=10 --pretrain_steps=4681 --resnet_depth=18 --model_dir=/kaggle/working/trained_model2 --eval_split=test --use_tpu=False

* 10epochのfine-tuning
* acc=0.614
* supervisedデータの割合が10%なのが問題設定として難しすぎる可能性

* supervised ratio=50%の場合を試してみる
* supervised ratioはpretrainには関係ないので、すでに40epoch学習済みのモデルを使用する

In [ ]:
#! cp -r ../input/trained-model/trained_model /kaggle/working/trained_model

In [ ]:
#! python /kaggle/working/simclr/tf2/run.py --learning_rate=1e-3 --optimizer=adam --mode=train_then_eval --train_mode=finetune --train_batch_size=32 --train_epochs=10 --supervised_ratio=0.5 --pretrain_steps=19188 --resnet_depth=18 --model_dir=/kaggle/working/trained_model --eval_split=test --use_tpu=False

* lr=1e-3
* supervised_ratio=0.5
* epochs=10
* 上記の設定でfinetuningしてaccは0.628(supervised_ratio=0.1の時と変わってない)

# 1/20

70epoch学習したモデルでfine-tuningをしてみる。

事前学習時の性能は、
* contrast_acc: 0.815
* supervised_acc: 0.628

In [ ]:
#! git clone https://github.com/ta9ryuWalrus/simclr.git
#! cp -r ../input/trained-model/trained_model /kaggle/working/trained_model
#! cp -r ../input/trained-model/trained_model /kaggle/working/trained_model70epochs # 古いやつはここに保管する

In [ ]:
#! python /kaggle/working/simclr/tf2/run.py --learning_rate=1e-3 --optimizer=adam --mode=train_then_eval --train_mode=finetune --train_batch_size=32 --train_epochs=10 --pretrain_steps=33228 --resnet_depth=18 --model_dir=/kaggle/working/trained_model --eval_split=test --use_tpu=False

70epochの事前学習の後、10epochのfinetuningをした結果
* accuracy: 0.639

# 画像サイズを小さくする(600->400)

In [ ]:
#! git clone https://github.com/ta9ryuWalrus/simclr.git
#! cp -r ../input/trained-model/trained_small_model /kaggle/working/trained_small_model

In [ ]:
#!python /kaggle/working/simclr/tf2/run.py --learning_rate=1e-3 --optimizer=adam --train_mode=pretrain --train_batch_size=64 --train_epochs=40 --resnet_depth=18 --model_dir=/kaggle/working/trained_small_model --eval_split=test --use_tpu=False

40epochの事前学習
* contrast_acc = 0.641
* supervised_acc = 0.630

In [ ]:
#! python /kaggle/working/simclr/tf2/run.py --learning_rate=1e-3 --optimizer=adam --mode=train_then_eval --train_mode=finetune --train_batch_size=64 --train_epochs=10 --pretrain_steps=9360 --resnet_depth=18 --model_dir=/kaggle/working/trained_small_model --eval_split=test --use_tpu=False

40epochの事前学習->10epochのfine-tuning
* accuracy = 0.643

# 1/25
k-foldで評価

In [ ]:
! git clone https://github.com/ta9ryuWalrus/simclr.git

* seed=0
* 5-foldの1つめ
* この設定だと20epochくらいでcontrast accが92%くらいに収束する

In [ ]:
#! python /kaggle/working/simclr/tf2/run.py --learning_rate=1e-4 --seed=0 --fold=4 --optimizer=adam --train_mode=pretrain --train_batch_size=64 --train_epochs=40 --resnet_depth=18 --model_dir=/kaggle/working/trained_model_seed0_fold4 --eval_split=test --use_tpu=False

In [ ]:
! cp -r ../input/trained-model/trained_model_seed0_fold4 /kaggle/working/trained_model_seed0_fold4

In [ ]:
! python /kaggle/working/simclr/tf2/run.py --learning_rate=1e-4 --seed=0 --fold=4 --optimizer=adam --mode=train_then_eval --train_mode=finetune --train_batch_size=64 --train_epochs=40 --pretrain_steps=10947 --resnet_depth=18 --model_dir=/kaggle/working/trained_model_seed0_fold4 --eval_split=test --use_tpu=False

10epochのfinetuningで
* loss 0.00245
* acc 0.614

## 出力のチェック

In [ ]:
! python /kaggle/working/simclr/tf2/run.py --learning_rate=1e-4 --seed=0 --fold=4 --optimizer=adam --mode=check --train_batch_size=64 --resnet_depth=18 --model_dir=/kaggle/working/trained_model_seed0_fold4 --eval_split=test --use_tpu=False

In [ ]:
! rm -r /kaggle/working/simclr

## データの分布をチェック

In [ ]:
'''
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold

df = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for train_idx, test_idx in kf.split(df['image_id'], df['label']):
  train_df = df.iloc[train_idx]
  test_df = df.iloc[test_idx]
  train_labels = np.array(train_df['label'])
  labeled_idx, _ = train_test_split(train_labels, train_size=0.1, random_state=0)
  labeled_df = train_df.iloc[labeled_idx]
  print()
  print('labeled train set:')
  print(labeled_df['label'].value_counts())
  print()
  print('test set:')
  print(test_df['label'].value_counts())
'''